In [2]:

import sys
import os

sys.path.append("..")

from paddleocr import PaddleOCR
import cv2
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
from vid2frames import convert2frames

In [5]:
ocr = PaddleOCR(use_angle_cls=True, lang='en', show_log = False) # need to run only once to download and load model into memory

In [3]:
# First run vid2frames on the video I'll be dugging OCR on

def reset_frames(input_path: str, output_folder: str):
    if os.path.exists(output_folder):
        os.system(f"rm -rf {output_folder}")
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
        os.mkdir(output_folder + "/images")
    convert2frames(input_path, output_folder + "/images")


100%|██████████| 578/578 [00:45<00:00, 12.73it/s]


In [6]:
def tensor_to_cv2(tens):
    as_numpy = tens.detach().numpy()
    as_numpy = as_numpy.swapaxes(0, 1)
    as_numpy = as_numpy.swapaxes(1, 2)
    as_numpy = as_numpy * 255
    return as_numpy.astype(np.uint8)

In [ ]:
def prepare_index(
    f_loader: DataLoader,
    out_dir: str,
):
    for ix, (images, labels) in tqdm(enumerate(f_loader), total=len(f_loader)):
        rgb = tensor_to_cv2(images[0])
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        results = ocr.ocr(rgb, cls=True)
        print(results)
        

In [8]:
def preprocess(
    inp_file: str,
    out_dir: str,
    conf_threshold=0.8,
    make_frames=True
):
    identifier = inp_file.split("/")[-1].split(".")[0]
    frame_dir = f"../frames/{identifier}"
    if make_frames:
        reset_frames(inp_file, frame_dir)
    f_dataset = datasets.ImageFolder(root=frame_dir, transform=transforms.ToTensor())
    f_loader = DataLoader(f_dataset, batch_size=1, shuffle=False)
    prepare_index(f_loader, out_dir)

preprocess("../videos/artisanal/play_ninety.mp4", "test", make_frames=False)